In [1]:
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
cwd = os.getcwd()


C:\Users\15U560\Anaconda3\Scripts\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## LOAD DATA

In [2]:
cwd = os.getcwd()
loadpath = cwd + "/data/custom_data.npz"
l = np.load(loadpath)
print (l.files)

['trainimg', 'trainlabel', 'testimg', 'testlabel', 'imgsize', 'use_gray', 'categories']


In [3]:
os.getcwd()

'C:\\Users\\15U560\\Desktop'

## PARSE LOADED DATA

In [4]:
trainimg = l['trainimg']
trainlabel = l['trainlabel']
testimg = l['testimg']
testlabel = l['testlabel']
imgsize = l['imgsize']
categories = l['categories']
use_gray = l['use_gray']
if use_gray:
    nimgch = 1
else:
    nimgch = 3
ntrain = trainimg.shape[0]
nclass = trainlabel.shape[1]
dim = trainimg.shape[1]
ntest = testimg.shape[0]
print ("%d TRAIN IMAGES" % (ntrain))
print ("%d TEST IMAGES" % (ntest))
print ("%d DIMENSIONAL INPUT" % (dim))
print ("%d CLASSES" % (nclass))
print (imgsize)
print (categories)

234 TRAIN IMAGES
156 TEST IMAGES
12288 DIMENSIONAL INPUT
5 CLASSES
[64 64]
['Jui' 'Momonogi' 'Jordan' 'Jobs' 'Jae seok']


## NETWORK TOPOLOCIES

In [5]:
n_input = dim
n_channel = 64
n_classes = nclass

In [6]:
# INPUTS AND OUTPUTS
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

# NETWORK PARAMETERS
stddev = 0.1
weights = {
    'c1' : tf.Variable(tf.random_normal([7, 7, nimgch, n_channel], stddev=stddev)),
    'd1' : tf.Variable(tf.random_normal([n_channel*imgsize[0]*imgsize[1]//4, n_classes], stddev=stddev))
}
biases = {
    'c1' : tf.Variable(tf.random_normal([n_channel], stddev=stddev)),
    'd1' : tf.Variable(tf.random_normal([n_classes], stddev=stddev))
}
print ("NETWORK READY")

NETWORK READY


In [7]:
# MODEL
def CNN(_x, _w, _b):
    # RESHAPE
    _x_r = tf.reshape(_x, shape=[-1, imgsize[0], imgsize[1], nimgch])
    # CONVOLUTION
    _conv1 = tf.nn.conv2d(_x_r, _w['c1'], strides=[1, 1, 1, 1], padding='SAME')
    # ADD BIAS
    _conv2 = tf.nn.bias_add(_conv1, _b['c1'])
    # RELU
    _conv3 = tf.nn.relu(_conv2)
    # MAX-POOL
    _pool = tf.nn.max_pool(_conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # VECTORIZE
    _dense = tf.reshape(_pool, [-1, _w['d1'].get_shape().as_list()[0]])
    # DENSE
    _logit = tf.add(tf.matmul(_dense, _w['d1']), _b['d1'])
    _out = {
            'x_r' : _x_r, 'conv1' : _conv1, 'conv2' : _conv2, 'conv3' : _conv3, 
                    'pool' : _pool, 'dense' : _dense, 'logit' : _logit
    }
    return _out

In [29]:
# PREDICTION
cnnout = CNN(x, weights, biases)

# LOSS AND OPTIMIZER
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=cnnout['logit']))
optm = tf.train.AdamOptimizer(learning_rate=3e-2).minimize(cost)
corr = tf.equal(tf.argmax(cnnout['logit'], 1), tf.argmax(y, 1))
accr = tf.reduce_mean(tf.cast(corr, "float"))

# INITIALIZER
init = tf.global_variables_initializer()
print ("FUNCTIONS READY")

FUNCTIONS READY


## RUN

In [30]:
# PARAMETERS
training_epochs = 50
batch_size = 100
disp_each = 10
plot_each = 10
# LAUNCH THE GRAPH
sess = tf.Session()
sess.run(init)
  

In [31]:
# OPTIMIZE
for epoch in range(training_epochs):
    total_batch = int(ntrain/batch_size)
    randpermlist = np.random.permutation(ntrain)
    sum_cost = 0.
    for i in range(total_batch):
        randidx = randpermlist[i*batch_size:min((i+1)*batch_size, ntrain-1)]
        batch_xs = trainimg[randidx, :]
        batch_ys = trainlabel[randidx, :]
        feeds = {x: batch_xs, y: batch_ys}
        sess.run(optm, feed_dict=feeds)
        sum_cost += sess.run(cost, feed_dict=feeds)
    avg_cost = sum_cost / total_batch
    
    if (epoch+1) % disp_each == 0 or epoch == training_epochs-1:
        print ("Epoch: %03d/%03d cost : %.9f" % (epoch+1, training_epochs, avg_cost))
        feeds = {x: batch_xs, y:batch_ys}
        train_acc = sess.run(accr, feed_dict=feeds)
        print (" TRAIN ACCURACY : %.3f" % (train_acc))
        feeds = {x: testimg, y:testlabel}
        test_acc = sess.run(accr, feed_dict=feeds)
        print ("TEST ACCURACY : %.3f" % (test_acc))

KeyboardInterrupt: 

In [ ]:
 #PLOT
    if (epoch+1) % plot_each == 0 or epoch == training_epochs-1:
        ntemp = int(ntest/batch_size) + 1
        missidxlist = []
        for ii in range(ntemp):
            iii = np.arange(ii*batch_size, min(ntest, (ii+1)*batch_size))
            feeds = {x: testimg[iii, :], y: testlabel[iii, :]}
            missidx = np.where(sess.run(corr.feed_dict=feeds) == 0)[0]
            missidxlist = np.concatenate99missidxlist, iii[missidx]
        nmiss = len(missidxlist)
        print ("%d MISSED" % (nmiss))
        # PLOT MISSCLASSIFIED (UPTO THREE)
        missidxlist = np.random.permutation(missidxlist.astype(int))
        selmissidx = missidxlist[:min(3, nmiss)].astype(int)
        feeds_miss = sess.run(pred, feed_dict=feeds_miss)
        answ_miss = np.argmax(pred_miss, 1)
        answ_corr = np.argmax(testlabel[selmissidx, :] ,1)
        for idx in range(len(selmissidx)):
            cidx = selmissidx[idx]
            anw = answ_mill[idx]
            ref = answ_corr[idx]
            if use_gray != 0:
                currimg = np.reshape(testimg[cidx, :], (imgsize[0], imgsize[1]))
                plt.imshow(currimg, cmap='gray')
            else :
                currimg = np.reshape(testimg[cidx, :], (imgsize[0], imgsize[1], -1))
                plt.imshow(currimg)
            anw_category = categories[anw]
            ref_category = categories[ref]
            title_string = ("IDX: %d CURR ANWR: [%s] TRUE LABEL: [%s]"
                    % (cidx, anw_category, ref_category))
            plt.title(title_string)
            plt.show()